In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

## Load data from sql loss_profit.db

In [4]:
# Load database .db from pipelines/data_engineer/database/loss_profit.db
from sqlalchemy import create_engine, text
from pathlib import Path

CURR_DIR = Path.cwd()
ROOT_DIR = CURR_DIR.parent.parent
DB_PATH = ROOT_DIR / 'data_engineer' / 'database' / 'loss_profit.db'

# Load data base .db and check tables
engine = create_engine(f'sqlite:///{DB_PATH}')
with engine.connect() as connection:
    result = connection.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    tables = result.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

if tables:
    for table_name in tables:
        with engine.connect() as connection:
            query = text(f"SELECT * FROM {table_name[0]}")
            df = pd.read_sql(query, connection)
            print(f"\nData from table '{table_name[0]}':")
            print(f"Database: {df.head()}")
else:
    print("No tables found in the database.")

Tables in the database:
loss_profit

Data from table 'loss_profit':
Database:      item_id  purchase_count  price  stocks  total_revenue  stock_value  \
0  item_0001              34   88.0      98         2992.0       8624.0   
1  item_0002              37  116.0      55         4292.0       6380.0   
2  item_0003              49  103.0      46         5047.0       4738.0   
3  item_0004              21   59.0      86         1239.0       5074.0   
4  item_0005              49   97.0      76         4753.0       7372.0   

  profit_status  
0        profit  
1        profit  
2        profit  
3        profit  
4        profit  


## Database as df -> Transformed database by data_engineer dir

### - **Added new column on total_revenue, stock_value, profit_status**

In [10]:
df

,item_id,purchase_count,price,stocks,total_revenue,stock_value,profit_status
0,item_0001,34,88.0,98,2992.0,8624.0,profit
1,item_0002,37,116.0,55,4292.0,6380.0,profit
2,item_0003,49,103.0,46,5047.0,4738.0,profit
3,item_0004,21,59.0,86,1239.0,5074.0,profit
4,item_0005,49,97.0,76,4753.0,7372.0,profit
...,...,...,...,...,...,...,...
8620,item_8621,14,23.0,41,322.0,943.0,profit
8621,item_8622,4,119.0,18,476.0,2142.0,profit
8622,item_8623,42,36.0,75,1512.0,2700.0,profit
8623,item_8624,31,34.0,90,1054.0,3060.0,profit
